# < Basic Recommendation Algorithm >

---

## 1. Popularity, High Rated Based

* 가장 단순함 => 평점이 높은 것을 추천
* 모두에게 같은 item 추천

In [1]:
ratings = {
     'doky':{'Anne_with_an_E':5,'Strange_Things':3,'The_Escape':3},
     'steve':{'Anne_with_an_E':5,'Strange_Things':1,'The_Escape':4},
     'jamie':{'Anne_with_an_E':0,'Strange_Things':4,'The_Escape':5},
     'bobby':{'Anne_with_an_E':2,'Strange_Things':1,'The_Escape':5}
}

In [12]:
recmd_movielist = dict()

# (A)
for name in ratings:
    for title in ratings[name]:
        score = ratings[name][title]
        
        if title not in recmd_movielist:
            recmd_movielist[title] = score
        else:
            recmd_movielist[title] += score

#



# (B)
for recmd_movietitle in recmd_movielist:
    recmd_movielist[recmd_movietitle] /= 4
#



# (C)
from operator import itemgetter
recmd_movielist = sorted(recmd_movielist.items(), reverse=True, key=itemgetter(1))
#

recmd_movielist

[('The_Escape', 4.25), ('Anne_with_an_E', 3.0), ('Strange_Things', 2.25)]

> 네 사람이 같은 영화에 준 평점들을 각각 더한 다음`(A)`<br>4로 나눠서 각 영화가 받은 평점의 평균을 낸다`(B)`<br>그 평균으로 내림차순 정렬해서 추천 리스트를 만든다`(C)`

---

## 2. Collaborative Filtering

* 사용자가 입력한 선호도(평점)를 사용하여 사용자-항목 선호도(평점) 행렬 계산
* 그 행렬을 사용하여 사용자들 간의 유사도를 계산
* 사용자들 간의 유사도를 바탕으로 모든 항목에 대해 예측 값을 계산하고, 높은 예측 값을 갖는 상위 N개의 추천 목록을 생성

In [13]:
ratings = {
     'doky':{'Anne_with_an_E':5,'Strange_Things':3,'The_Escape':3},
     'steve':{'Anne_with_an_E':5,'Strange_Things':1,'The_Escape':4},
     'jamie':{'Strange_Things':4,'The_Escape':5},
     'bobby':{'Anne_with_an_E':2,'Strange_Things':1,'The_Escape':5}
}

In [17]:
import math

def sim_w_dist(i, j):
    return math.sqrt(pow(i,2), pow(j,2))

> 피타고라스 공식을 통한 거리 계산으로 유사도 측정

---

# Reference

* https://www.fun-coding.org/recommend_basic2.html